In [1]:
import pandas as pd

train_df = pd.read_csv('csv/train.csv')

In [2]:
print(train_df.shape)
train_df.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
"""
(遊び)

nullの列を除いたデータセット
カテゴリ変数のみのデータセット

"""

no_null_df = train_df.loc[:, ~train_df.isnull().any()]
obj_df = train_df.loc[:, train_df.dtypes == 'object']


## One-Hotエンコーディング

In [ ]:
"""
・数値の大小関係を各変数に持たせない -> 線形回帰やニューラルネットに有効
・カテゴリ数が多いと特徴量を選択するアルゴリズム（ランダムフォレストなど）では数値変数の選択率が減少してしまう。
 　→PCA, Embeddingで次元削減
"""

In [4]:
print(train_df['Condition1'].value_counts())

#↓いらない？？？
# train_oh = pd.DataFrame()
train_oh = pd.get_dummies(train_df['Condition1'], prefix='Condition1', prefix_sep='_')
print(train_oh.shape)
train_oh.head()


Norm      1260
Feedr       81
Artery      48
RRAn        26
PosN        19
RRAe        11
PosA         8
RRNn         5
RRNe         2
Name: Condition1, dtype: int64
(1460, 9)


,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn
0,0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0


## ダミーコーディング

In [ ]:
"""
・one-hotとの違い→生成する特量量が一つ少ない
　→多重共線性を回避できる
・カテゴリかる変数の要素の数だけカラムが増えて、メモリの消費が大きくなる

"""

In [246]:
"""
【疑問点】

デメリットとして、カテゴリカル変数の要素の数だけカラムが増え、スパースなデータとなり、結果、メモリを消費が大きくなる。
とあるが、これはone-hotコーディングも同様では？？

"""

'\n【疑問点】\n\nデメリットとして、カテゴリカル変数の要素の数だけカラムが増え、スパースなデータとなり、結果、メモリを消費が大きくなる。\nとあるが、これはone-hotコーディングも同様では？？\n\n'

In [6]:
print(train_df['Condition1'].value_counts())


train_oh = pd.get_dummies(train_df['Condition1'], prefix='Condition1', prefix_sep='_', drop_first=True)
print(train_oh.shape)
train_oh.head()


Norm      1260
Feedr       81
Artery      48
RRAn        26
PosN        19
RRAe        11
PosA         8
RRNn         5
RRNe         2
Name: Condition1, dtype: int64
(1460, 8)


,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn
0,0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0


## Effectコーディング

In [7]:
"""
参照カテゴリを-1で表現
全体の平均との比較で使用？？
"""

'\n参照カテゴリを-1で表現\n全体の平均との比較で使用？？\n'

In [8]:
train_ef = train_oh.copy()

#whereは条件がTrueの場合->そのまま、 Falseの場合->第二引数に置換
train_ef['Condition1_Norm'] = train_ef['Condition1_Norm'].where(train_ef['Condition1_Feedr'] == 0, -1)

train_ef[['Condition1_Feedr', 'Condition1_Norm']].head(10)

,Condition1_Feedr,Condition1_Norm
0,0,1
1,1,-1
2,0,1
3,0,1
4,0,1
5,0,1
6,0,1
7,0,0
8,0,0
9,0,0


## ラベル・エンコーディング

In [ ]:
"""
・ラベルの種類に応じた数値の割り当てをする。(1, 2,3・・・)
・離散値になるので決定木、ランダムフォレストと相性が良い（データの区画分け（分類）だから？）
・係数、特徴量の線形関係となる線形回帰では有効ではない（割り振られた数値に大小の意味はないから？）

メリット：
カラム数はそのまま

デメリット：
　欠損値に対応しない　
　ラベル間の数値の大小関係を学習してしまう。
　無意味な順序付けが行われる　→　名義尺度は避ける
　順位間の差が一定でないものは、エンコーディングで一定に変換されてしまう

"""

In [9]:
from sklearn.preprocessing import LabelEncoder

print(train_df['ExterQual'].value_counts())
train_lb = train_df.copy()

lbl = LabelEncoder()
train_lb['ExterQual_enc'] = lbl.fit_transform(train_lb['ExterQual'])
train_lb[['ExterQual', 'ExterQual_enc']] 

TA    906
Gd    488
Ex     52
Fa     14
Name: ExterQual, dtype: int64


,ExterQual,ExterQual_enc
0,Gd,2
1,TA,3
2,Gd,2
3,TA,3
4,Gd,2
...,...,...
1455,TA,3
1456,TA,3
1457,Ex,0
1458,TA,3


## Ordinalエンコーディング

In [ ]:
""""""

In [10]:
map_dict = {'TA': 0, 'Gd': 1, 'Ex': 2, 'Fa': 3}

def map_values(x):
    return map_dict[x]

train_oe = train_df.copy()
train_oe['ExterQual_oe'] = train_oe['ExterQual'].apply(lambda x: map_dict[x])
train_oe[['ExterQual', 'ExterQual_oe']].tail()


,ExterQual,ExterQual_oe
1455,TA,0
1456,TA,0
1457,Ex,2
1458,TA,0
1459,Gd,1


In [ ]:
"""
OrdinalEncoderがあったなと思い出し、下のコードを自分で実装

"""

In [32]:
from sklearn.preprocessing import OrdinalEncoder

train_od = train_df.copy()

# categories='auto'の場合
od = OrdinalEncoder() 
train_od_enc = od.fit_transform(train_od['ExterQual'].values.reshape(-1, 1))
train_od_df = pd.DataFrame(train_od_enc)
train_od_df.columns = ['ExterQual_od']


# categories=指定した場合
od_cat = OrdinalEncoder(categories=[['TA', 'Gd', 'Ex', 'Fa']])
train_od_cat = od_cat.fit_transform(train_od['ExterQual'].values.reshape(-1, 1))
train_od_cat_df = pd.DataFrame(train_od_cat)
train_od_cat_df.columns = ['ExterQual_od_cat']

# 結合したデータフレーム[オリジナル、引数なしのord、 引数ありのord]
df_2 =  train_od_df.join(train_od_cat_df)
df_all = train_od.loc[:, ['ExterQual']].join(df_2)

df_all.tail()

,ExterQual,ExterQual_od,ExterQual_od_cat
1455,TA,3.0,0.0
1456,TA,3.0,0.0
1457,Ex,0.0,2.0
1458,TA,3.0,0.0
1459,Gd,2.0,1.0


In [37]:
 """
 エンコーディングのページのコード
 （OrdinalEncoderのコードもあることを気づかなかった。）
 """
    
train_oe = train_df.copy()
oe = OrdinalEncoder()
train_oe[['外部の品質_oe', '外部の条件_oe']] = oe.fit_transform(train_df[['ExterQual', 'ExterCond']]) # 欠損値不可
train_oe[['ExterQual', '外部の品質_oe', 'ExterCond', '外部の条件_oe']].tail()

print(train_df['ExterQual'].value_counts())
print(train_oe['外部の品質_oe'].value_counts())
print('='*30)
print(train_df['ExterCond'].value_counts())
print(train_oe['外部の条件_oe'].value_counts())
    

TA    906
Gd    488
Ex     52
Fa     14
Name: ExterQual, dtype: int64
3.0    906
2.0    488
0.0     52
1.0     14
Name: 外部の品質_oe, dtype: int64
TA    1282
Gd     146
Fa      28
Ex       3
Po       1
Name: ExterCond, dtype: int64
4.0    1282
2.0     146
1.0      28
0.0       3
3.0       1
Name: 外部の条件_oe, dtype: int64


## Feature hashing

In [ ]:
"""
カテゴリ変数 -> ハッシュ関数 -> One-Hotエンコーディング

メリット：
・変換前のカテゴリ変数の水準数より、変換後の特徴量の数を少なくすることができる。
　　→one-hotで特徴量が膨大になる場合、使用する
・メモリの節約
 
デメリット：
・衝突の可能性
・データ探索、可視化に不向き
"""

In [80]:
from sklearn.feature_extraction import FeatureHasher
from sys import getsizeof

tgt = train_df['OverallQual'].astype('str')
# print(tgt.value_counts())
m = len(tgt.unique())
print(f'要素数（次元数）：　{m}')


#二次元化しないとfitできない（参考コードに記載なし、エラーが出る）
tgt_2d = tgt.values.reshape(-1,1)
h = FeatureHasher(n_features=m, input_type='string')
f = h.fit_transform(tgt_2d)

"""
【疑問】

下のコメントについて、
元々が疎行列ならndarrayに変換しても疎行列なのでは？
"""
# 疎行列（スパース行列）を、NumPy配列 ndarray（密行列（非スパース行列）） に変換
f.toarray()

# todense()でndarrayに変換
df = pd.DataFrame(data = f.todense().astype(str), columns = ['ハッシュ{}'.format(x+1) for x in range(m)])
df.head()

# 変換前後のデータ容量の確認
print(f'メモリサイズ（前）： {getsizeof(tgt)}')
print(f'メモリサイズ（後）： {getsizeof(f)}')

要素数（次元数）：　10
メモリサイズ（前）： 84842
メモリサイズ（後）： 48


## カウント（Frequency）エンコーディング

In [ ]:
"""
特徴量の各値の出現頻度で置き換える
エンコーディングの値の大小に意味（出現頻度）がある

メリット：
・次元を増やさない

デメリット：
・外れ値に敏感　->　対数変換すれば弱められる
・異なるカテゴリカル変数に同じ値が割り当てられてしまう場合がある

"""

In [100]:
count_df = train_df.copy()
count_map = count_df['OverallQual'].value_counts().to_dict()

#map()の引数にdictを与えると対応するキーの値に置換する
count_df['OverallQual_cnt'] = count_df['OverallQual'].map(count_map)
count_df[['OverallQual','OverallQual_cnt']].head()

,OverallQual,OverallQual_cnt
0,7,319
1,6,374
2,7,319
3,7,319
4,8,168


## ラベル・カウント（Ranked frequency）エンコーディング

In [ ]:
"""
特徴量の各値の出現頻度で順位付けて、その順位に置き換える

メリット：
・データの間隔が1になり、外れ値の影響が小さくなる

デメリット：
・同じ出現頻度のカテゴリカル変数に無意味な順序が割り当てられる

"""

In [124]:
#rank()はデフォルトで昇順 => 出現頻度が少ないものから順位付け
rank_df = train_df.copy()
rank_map = rank_df['OverallQual'].value_counts().rank(ascending=False).to_dict()

rank_df['OverallQual_cut'] = rank_df['OverallQual'].map(rank_map)
rank_df[['OverallQual', 'OverallQual_cut']].head()

,OverallQual,OverallQual_cut
0,7,3.0
1,6,2.0
2,7,3.0
3,7,3.0
4,8,4.0


## ビンカウンティング

In [ ]:
"""
カテゴリ変数をカテゴリに関数る統計量に置換
・目的変数に対して、有用な集計量を設定する必要がある
・交互作用項の生成の際に利用できる

メリット：
・統計量の取り方次第で次元数を減らせる

デメリット：
・目的変数を使用した統計量を学習したい場合、データ分割（Fold）をする必要がある

"""

In [202]:
bin_count = train_df.copy()
bin_df = pd.DataFrame()

target = bin_count.groupby('MSSubClass').agg({'Fireplaces':'mean'})

#元々のFireplaceは'Fireplace_x'でグループ化した値は'Fireplace_y' =>結合して特徴名が被るから？
#参考コードではright_index='MSSubClass'をしているが、onとright_indexの両方を指定できない
bin_df = pd.merge(bin_count, target, on='MSSubClass')[['MSSubClass','Fireplaces_y']].astype(str)
bin_df.head()

,MSSubClass,Fireplaces_y
0,60,0.8862876254180602
1,60,0.8862876254180602
2,60,0.8862876254180602
3,60,0.8862876254180602
4,60,0.8862876254180602


In [194]:
#結合後のグループ化した値のカラム名を検索
col_list = [ i for i in bin_count.columns if i != 'Fireplaces']
bin_df.drop(columns = col_list, axis=1)

,Fireplaces_x,Fireplaces_y
0,0,0.886288
1,1,0.886288
2,1,0.886288
3,2,0.886288
4,2,0.886288
...,...,...
1455,0,0.000000
1456,0,0.500000
1457,0,0.500000
1458,1,0.500000


In [160]:
#参照ページで使われている'暖房'のカラムは何？の検索
bin_count.dtypes[(bin_count.dtypes=='int64') | (bin_count.dtypes== 'float64') ]

Id                 int64
MSSubClass         int64
LotFrontage      float64
LotArea            int64
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
MasVnrArea       float64
BsmtFinSF1         int64
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
1stFlrSF           int64
2ndFlrSF           int64
LowQualFinSF       int64
GrLivArea          int64
BsmtFullBath       int64
BsmtHalfBath       int64
FullBath           int64
HalfBath           int64
BedroomAbvGr       int64
KitchenAbvGr       int64
TotRmsAbvGrd       int64
Fireplaces         int64
GarageYrBlt      float64
GarageCars         int64
GarageArea         int64
WoodDeckSF         int64
OpenPorchSF        int64
EnclosedPorch      int64
3SsnPorch          int64
ScreenPorch        int64
PoolArea           int64
MiscVal            int64
MoSold             int64
YrSold             int64
SalePrice          int64
dtype: object

## Nonエンコーディング



In [ ]:
"""
Nan値がある場合、そのダミー変数を作る

メリット：
・欠損値に意味がある場合は効果的

デメリット：
・意味がない欠損値にはノイズが多く学習し、精度の低下につながる

"""

In [244]:
miss_val =  train_df.copy()

#参考コードではT(転置)していたが、必要ないし、このコードは0番目の行の値の抽出になる
# num = miss_val['BsmtCond'].T.values[0]

#このコードが正しいのでは？？(自分で実装)
num = miss_val['BsmtCond'].isnull().sum()
print(f'欠損値の数：　{num}')

print(miss_val['BsmtCond'].value_counts(dropna=False))


train_non = pd.get_dummies(train_df['BsmtCond'], prefix='BsmCond', prefix_sep='_', drop_first=True, dummy_na=True)
train_non.head(10)

欠損値の数：　37
TA     1311
Gd       65
Fa       45
NaN      37
Po        2
Name: BsmtCond, dtype: int64


,BsmCond_Gd,BsmCond_Po,BsmCond_TA,BsmCond_nan
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,1,0,0,0
4,0,0,1,0
5,0,0,1,0
6,0,0,1,0
7,0,0,1,0
8,0,0,1,0
9,0,0,1,0
